In [15]:
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tabulate import tabulate
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_validate

# Парсим

In [16]:
# Запускалось локально, здесь решил закомментировать

# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.by import By
# from selenium import webdriver
# from selenium.common import NoSuchElementException
# from selenium.webdriver.chrome.service import Service
# from selenium.common import ElementClickInterceptedException

def get_laptop_data(_driver):
    html = _driver.page_source
    expand_button = driver.find_element(By.CLASS_NAME, "product-characteristics__expand")
    actions = ActionChains(driver)
    actions.move_to_element(expand_button).perform()
    expand_button.click()
    time.sleep(1)
    soup = BeautifulSoup(html)
    laptop_data = {}
    for param_data in soup.find_all('div', {"class": "product-characteristics__spec"}):
        param = param_data.find('div', {"class": "product-characteristics__spec-title"}).text.strip()
        value = param_data.find('div', {"class": "product-characteristics__spec-value"}).text.strip()
        laptop_data[param] = value
    return laptop_data

def force_get(_driver, _url):
    _driver.get(_url)
    while '404' in _driver.title:
        _driver.get(_url)

def parse():
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
  url = 'https://www.dns-shop.ru/catalog/17a892f816404e77/noutbuki/'
  driver.get(url)
  driver.page_source
  wait = WebDriverWait(driver, 250)
  laptop_df = pd.DataFrame()
  for i in range(1, 12):
      force_get(driver, url)
      wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "city-select__text_BTU")))
      driver.find_element(By.CLASS_NAME, "city-select__text_BTU").click()

      wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "city-bubble_IBz")))
      city_button = driver.find_elements(By.CLASS_NAME, "city-bubble_IBz")[i]
      city_name = city_button.text.strip()
      wait.until(EC.element_to_be_clickable(city_button))
      city_button.click()

      page_number = 1
      while True:
          cur_page_url = f'https://www.dns-shop.ru/catalog/17a892f816404e77/noutbuki/?p={page_number}'
          force_get(driver, cur_page_url)
          print(f"City: {city_name}, page #{page_number}")

          laptop_links = driver.find_elements(By.CLASS_NAME, "catalog-product__name")
          if len(laptop_links) == 0:
              break

          laptop_links_count = len(laptop_links)
          time.sleep(1)
          wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "product-buy__price")))
          prices = [elem.text for elem in driver.find_elements(By.CLASS_NAME, "product-buy__price")]
          for laptop_count in range(laptop_links_count):
              is_ok = False
              new_laptop_data = {}
              while not is_ok:
                  try:
                      force_get(driver, cur_page_url)
                      wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "catalog-product__name")))
                      laptop_links = driver.find_elements(By.CLASS_NAME, "catalog-product__name")
                      laptop_links[laptop_count].click()
                      get_laptop_data(driver)
                      new_laptop_data = get_laptop_data(driver)
                      new_laptop_data['Цена'] = prices[laptop_count]
                      new_laptop_data['Город'] = city_name
                      print(f"Ноутбук #{laptop_count}. Кол-во параметров: {len(new_laptop_data)}")
                      if len(new_laptop_data) > 8:
                          is_ok = True
                      else:
                          print('Набрал мало данных, пробую снова')
                  except (AttributeError, NoSuchElementException, ElementClickInterceptedException) as e:
                      print('EXCEPTION: ', e)
                      is_ok = False
              laptop_df = laptop_df.append(new_laptop_data, ignore_index=True)
          page_number += 1
  laptop_df.to_csv('dirty_laptops.csv')

  # parse()

# Обрабатываем

In [17]:
def handle_price(price_str: str) -> int:
    first_value = price_str.split('\n')[0]
    if first_value[-1] == '₽':
        first_value = first_value[:-1]
    return int(re.sub(r'\s+', '', first_value))

def to_float(string):
    return (float(str(string).split()[0]) if str(string) != 'нет' else 0.0) if not pd.isna(string) else pd.NA

def to_bool(value):
    return value != 'нет'

def apply(_df, func, columns_to_apply):
    for _col in columns_to_apply:
        _df[_col] = _df[_col].apply(func)

def save_clear_df(_laptop_df):
    _laptop_df = pd.read_csv('laptops_dirty.csv', index_col=0)
    _laptop_df = _laptop_df.replace(['12 мес.', '24 мес.', '36 мес.'], [1, 2, 3])
    _laptop_df['Диагональ экрана (дюйм)'] = _laptop_df['Диагональ экрана (дюйм)'].fillna(value='15.6"')
    _laptop_df = _laptop_df.drop(columns=['Диагональ экрана'])
    _laptop_df['Диагональ экрана (дюйм)'] = _laptop_df['Диагональ экрана (дюйм)'].apply(lambda x: float(str(x)[:-1]))
    _laptop_df['Цена'] = _laptop_df['Цена'].apply(handle_price)

    columns_to_bool = [
        'Сенсорный экран',
        'Игровой ноутбук',
        'Цифровой блок клавиатуры',
        'Сканер отпечатка пальца',
        'Механическая клавиатура',
        'Встроенный микрофон',
        'Оптический привод',
        'Поддержка USB Power Delivery',
        'Поддержка TPM'
    ]
    apply(_laptop_df, to_bool, columns_to_bool)

    columns_to_float = [
        'Максимальная частота обновления экрана',
        'Яркость',
        'Плотность пикселей',
        'Частота процессора',
        'Автоматическое увеличение частоты',
        'Объем оперативной памяти',
        'Общий объем твердотельных накопителей (SSD)',
        'Общий объем жестких дисков (HDD)',
        'Объем накопителя eMMC',
        'Веб-камера',
        'Приблизительное время автономной работы',
        'Глубина',
        'Ширина',
        'Толщина',
        'Вес',
        'Максимальный объем памяти',
        'Свободные слоты для оперативной памяти',
        'Выходная мощность адаптера питания',
        'Скорость вращения шпинделя',
        'Объем видеопамяти',
        'Тактовая частота с ускорением',
        'Максимальное энергопотребление',
        'Частота оперативной памяти'
    ]
    apply(_laptop_df, to_float, columns_to_float)

    _laptop_df.rename(columns={
        'Гарантия от производителя': 'Гарантия от производителя (лет)',
        'Сенсорный экран': 'Имеет сенсорный экран',
        'Цифровой блок клавиатуры': 'Есть цифровой блок клавиатуры',
        'Сканер отпечатка пальца': 'Есть Сканер отпечатка пальца',
        'Максимальная частота обновления экрана': 'Максимальная частота обновления экрана (Гц)',
        'Яркость': 'Яркость (Кд/м²)',
        'Плотность пикселей': 'Плотность пикселей (ppi)',
        'Частота процессора': 'Частота процессора (ГГц)',
        'Автоматическое увеличение частоты': 'Автоматическое увеличение частоты (ГГц)',
        'Объем оперативной памяти': 'Объем оперативной памяти (Гб)',
        'Частота оперативной памяти': 'Частота оперативной памяти (Мгц)',
        'Общий объем твердотельных накопителей (SSD)': 'Общий объем SSD (Гб)',
        'Общий объем жестких дисков (HDD)': 'Общий объем HDD (Гб)',
        'Объем накопителя eMMC': 'Объем накопителя eMMC (Гб)',
        'Веб-камера': 'Веб-камера (Мп)',
        'Встроенный микрофон': 'Есть встроенный микрофон',
        'Оптический привод': 'Есть оптический привод',
        'Поддержка USB Power Delivery': 'Есть поддержка USB Power Delivery',
        'Глубина': 'Глубина (мм)',
        'Ширина': 'Ширина (мм)',
        'Толщина': 'Толщина (мм)',
        'Вес': 'Вес (кг)',
        'Максимальный объем памяти': 'Максимальный объем памяти (Гб)',
        'Выходная мощность адаптера питания': 'Выходная мощность адаптера питания (Вт)',
        'Гарантия от продавца': 'Гарантия от продавца (лет)',
        'Скорость вращения шпинделя': 'Скорость вращения шпинделя (об/мин)',
        'Объем видеопамяти': 'Объем видеопамяти (Гб)',
        'Тактовая частота с ускорением': 'Тактовая частота с ускорением (Мгц)',
        'Максимальное энергопотребление': 'Максимальное энергопотребление (Вт)',
        'Приблизительное время автономной работы': 'Приблизительное время автономной работы (ч)'
    }, inplace=True)

    _laptop_df.to_csv('laptops.csv')

  # save_clear_df(pd.read_csv('dirty_laptops.csv'))

# Загружаем данные

In [18]:
!gdown 17z2iLnWzxcaFGHyurgxYhLMX2e3r0VWC

Downloading...
From: https://drive.google.com/uc?id=17z2iLnWzxcaFGHyurgxYhLMX2e3r0VWC
To: /content/laptops.csv
100% 7.39M/7.39M [00:00<00:00, 171MB/s]


In [19]:
df = pd.read_csv('laptops.csv', index_col=0)
df

,Гарантия от производителя (лет),Тип,Модель,Код производителя,Год релиза,Операционная система,Игровой ноутбук,Конструктивное исполнение,Цвет верхней крышки,Материал крышки,...,Цветовой охват,Гарантия от продавца (лет),Скорость вращения шпинделя (об/мин),Объем видеопамяти (Гб),Производитель видеочипа,Тактовая частота с ускорением (Мгц),Тип видеопамяти,Расширенные возможности,HDR,Максимальное энергопотребление (Вт)
0,1.0,ноутбук,Irbis NB77,[NB77],2021.0,Windows 10 Home Single Language,False,классический,черный,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,ноутбук,Irbis NB257,[NB257],2021.0,Windows 10 Home Single Language,False,классический,серый,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,ноутбук,Irbis NB283,[NB283],2021.0,Windows 10 Pro,False,классический,серый,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,ноутбук,DEXP Aquilon,[C14-ICW201],2022.0,Windows 11 Pro,False,классический,серебристый,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,ноутбук,DEXP Aquilon,[C14-ICW202],2022.0,Windows 11 Home Single Language,False,классический,серый,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999,1.0,ноутбук,Apple MacBook Pro,[MNWE3LL/A],2023.0,macOS,False,классический,серебристый,алюминий,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HDR10,NaN
8000,1.0,ноутбук,Apple MacBook Pro,[MNWA3LL/A],2023.0,macOS,False,классический,серый,алюминий,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HDR10,NaN
8001,1.0,ноутбук,MSI Raider GE78 HX 13VI-095RU,[9S7-17S111-095],2023.0,Windows 11 Home Single Language,True,классический,черный,пластик,...,NaN,NaN,NaN,16.0,NVIDIA,2040.0,GDDR6,"технология Resizable BAR, технология DLSS, ноу...",NaN,175.0
8002,2.0,ноутбук,MSI CreatorPro X17 A12UMS-205RU,[9S7-17Q121-205],2022.0,Windows 11 Pro,False,классический,черный,алюминий,...,NaN,NaN,NaN,16.0,NVIDIA,1640.0,GDDR6,NaN,NaN,175.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8004 entries, 0 to 8003
Data columns (total 86 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Гарантия от производителя (лет)              6139 non-null   float64
 1   Тип                                          8004 non-null   object 
 2   Модель                                       8004 non-null   object 
 3   Код производителя                            7520 non-null   object 
 4   Год релиза                                   7944 non-null   float64
 5   Операционная система                         8004 non-null   object 
 6   Игровой ноутбук                              8004 non-null   bool   
 7   Конструктивное исполнение                    8004 non-null   object 
 8   Цвет верхней крышки                          8004 non-null   object 
 9   Материал крышки                              7932 non-null   object 
 10  

# Визуализируем

In [21]:
px.box(
    df, 
    x="Вес (кг)", 
    title="<b>Статистики по весу</b>",
    labels={
        'Вес (кг)': '<b>Вес (кг)</b>'
    }
)

In [22]:
px.histogram(
    df[df['Цена'] < 250000], 
    color="Игровой ноутбук",
    x="Цена", 
    title="<b>Игровые ноутбуки дороже?</b>",
    barmode='overlay',
    labels={
        'Цена': '<b>Цена</b>',
        'Игровой ноутбук': 'Игровой ноутбук',
    }
)

In [23]:
px.pie(
    df.value_counts('Материал корпуса').reset_index(name='count'),
    title="<b>Материал корпуса</b>",
    values='count',
    names='Материал корпуса'
)

In [24]:
# category - category
data_to_show = df[df['Модель дискретной видеокарты'] != 'нет']
data_to_show = data_to_show[['Игровой ноутбук', 'Модель дискретной видеокарты']].value_counts().reset_index(name='count')
px.bar(
    data_to_show,
    x="Модель дискретной видеокарты",
    y='count',
    color='Игровой ноутбук',
    barmode='group',
    title='<b>Модели дискретных видокарт у игровых и неигровых ноутбуков</b>',
    labels={
        'Модель дискретной видеокарты': '<b>Модель дискретной видеокарты</b>',
        'Игровой ноутбук': 'Игровой ноутбук',
        'count': '<b>count</b>'
    }
)

In [25]:
# quantitative - quantitative
px.scatter(
    df,
    x='Яркость (Кд/м²)',
    y='Плотность пикселей (ppi)',
    title='<b>Зависимость яркости и плотности пикселей</b>',
    labels={
        'Яркость (Кд/м²)': '<b>Яркость (Кд/м²)</b>',
        'Плотность пикселей (ppi)': '<b>Плотность пикселей (ppi)</b>'
    }
)

In [26]:
# quantitative - category
px.box(
    df,
    color='Тип оперативной памяти',
    y='Цена',
    title='<b>Зависимость цены от типа оперативной памяти</b>',
    height=700,
    labels={
        'Цена': '<b>Цена (руб)</b>'
    }
)

# Обучаем модели

In [27]:
df

,Гарантия от производителя (лет),Тип,Модель,Код производителя,Год релиза,Операционная система,Игровой ноутбук,Конструктивное исполнение,Цвет верхней крышки,Материал крышки,...,Цветовой охват,Гарантия от продавца (лет),Скорость вращения шпинделя (об/мин),Объем видеопамяти (Гб),Производитель видеочипа,Тактовая частота с ускорением (Мгц),Тип видеопамяти,Расширенные возможности,HDR,Максимальное энергопотребление (Вт)
0,1.0,ноутбук,Irbis NB77,[NB77],2021.0,Windows 10 Home Single Language,False,классический,черный,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,ноутбук,Irbis NB257,[NB257],2021.0,Windows 10 Home Single Language,False,классический,серый,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,ноутбук,Irbis NB283,[NB283],2021.0,Windows 10 Pro,False,классический,серый,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,ноутбук,DEXP Aquilon,[C14-ICW201],2022.0,Windows 11 Pro,False,классический,серебристый,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,ноутбук,DEXP Aquilon,[C14-ICW202],2022.0,Windows 11 Home Single Language,False,классический,серый,пластик,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999,1.0,ноутбук,Apple MacBook Pro,[MNWE3LL/A],2023.0,macOS,False,классический,серебристый,алюминий,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HDR10,NaN
8000,1.0,ноутбук,Apple MacBook Pro,[MNWA3LL/A],2023.0,macOS,False,классический,серый,алюминий,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HDR10,NaN
8001,1.0,ноутбук,MSI Raider GE78 HX 13VI-095RU,[9S7-17S111-095],2023.0,Windows 11 Home Single Language,True,классический,черный,пластик,...,NaN,NaN,NaN,16.0,NVIDIA,2040.0,GDDR6,"технология Resizable BAR, технология DLSS, ноу...",NaN,175.0
8002,2.0,ноутбук,MSI CreatorPro X17 A12UMS-205RU,[9S7-17Q121-205],2022.0,Windows 11 Pro,False,классический,черный,алюминий,...,NaN,NaN,NaN,16.0,NVIDIA,1640.0,GDDR6,NaN,NaN,175.0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8004 entries, 0 to 8003
Data columns (total 86 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Гарантия от производителя (лет)              6139 non-null   float64
 1   Тип                                          8004 non-null   object 
 2   Модель                                       8004 non-null   object 
 3   Код производителя                            7520 non-null   object 
 4   Год релиза                                   7944 non-null   float64
 5   Операционная система                         8004 non-null   object 
 6   Игровой ноутбук                              8004 non-null   bool   
 7   Конструктивное исполнение                    8004 non-null   object 
 8   Цвет верхней крышки                          8004 non-null   object 
 9   Материал крышки                              7932 non-null   object 
 10  

## Обрабатываем данные для обучения

In [29]:
df_to_train = df[[
    'Год релиза',
    'Диагональ экрана (дюйм)',
    'Максимальная частота обновления экрана (Гц)',
    'Частота процессора (ГГц)',
    'Объем оперативной памяти (Гб)',
    'Общий объем SSD (Гб)',
    'Максимальный объем памяти (Гб)',
    'Цена',
    'Автоматическое увеличение частоты (ГГц)',
    'Максимальное число потоков'
]]

df_to_train = df_to_train[df_to_train['Цена'] <= 200000]
df_to_train.dropna(inplace=True)
df_to_train = df_to_train.convert_dtypes()
df_to_train = df_to_train.drop_duplicates()
df_to_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 1 to 7956
Data columns (total 10 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Год релиза                                   1014 non-null   Int64  
 1   Диагональ экрана (дюйм)                      1014 non-null   Float64
 2   Максимальная частота обновления экрана (Гц)  1014 non-null   Int64  
 3   Частота процессора (ГГц)                     1014 non-null   Float64
 4   Объем оперативной памяти (Гб)                1014 non-null   Int64  
 5   Общий объем SSD (Гб)                         1014 non-null   Int64  
 6   Максимальный объем памяти (Гб)               1014 non-null   Int64  
 7   Цена                                         1014 non-null   Int64  
 8   Автоматическое увеличение частоты (ГГц)      1014 non-null   Float64
 9   Максимальное число потоков                   1014 non-null   Int64  
dtype

In [30]:
fig = px.imshow(df_to_train.corr())
fig.update_layout(height=700)
fig

## Выбираем модель

In [31]:
from math import sqrt

df_with_dummies = pd.get_dummies(df_to_train)
X = df_with_dummies.drop(columns=['Цена'])
y = df_with_dummies['Цена']

models = [GradientBoostingRegressor(), LinearRegression(), RandomForestRegressor()]
metrics_data = [
    {'name': 'R²', 'sklearn_name': 'r2', 'calculator': r2_score}, 
    {'name': 'Root of MSE', 'sklearn_name': 'neg_root_mean_squared_error', 'calculator': mean_squared_error},
    {'name': 'MAE', 'sklearn_name': 'neg_mean_absolute_error', 'calculator': mean_absolute_error}
]
for model in models:
  cv_scores = cross_validate(
      model, X, y, cv=4, 
      scoring=[metric_data['sklearn_name'] for metric_data in metrics_data], 
      return_train_score=True
  )

  print(type(model).__name__)

  table_data = []
  for metric_data in metrics_data:
    scores = cv_scores[f"test_{metric_data['sklearn_name']}"]
    name = metric_data['name']
    table_data.append([name] + [f"{-s if s < 0 else s:.2f}" for s in scores])

  print(tabulate(table_data, headers=['Metric', '1', '2', '3', '4']))
  print()
  print()

GradientBoostingRegressor
Metric             1         2         3         4
-----------  -------  --------  --------  --------
R²              0.37      0.14      0.45      0.45
Root of MSE  8794.34  38610.6   26078.2   47040.1
MAE          6707.69  26912.2   17556.4   37058


LinearRegression
Metric              1         2         3         4
-----------  --------  --------  --------  --------
R²               0.25      0.05      0.48      0.55
Root of MSE  12358.9   37113.6   25374.9   48640.6
MAE           9401.1   26229.3   16960.7   38674.9


RandomForestRegressor
Metric              1        2         3         4
-----------  --------  -------  --------  --------
R²               0.4       0.3      0.38      0.35
Root of MSE  13081.7   41248.3  27617.4   45531.4
MAE           9680.44  30144.3  18684.4   35141.9




In [39]:
def evaluate_model(_model, _X_train, _X_test, _y_train, _y_test):
  _model.fit(_X_train, _y_train)
  _y_pred = _model.predict(_X_test)

  table_data = []
  for metric_data in metrics_data:
      name = metric_data['name']
      score = metric_data['calculator'](_y_test, _y_pred)
      score = sqrt(score) if name == 'Root of MSE' else score
      table_data.append([name, f"{score:.2f}"])
  print(tabulate(table_data, headers=['Metric', 'Score']))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=101)

# Оставил меньше значений для более быстрой работы
param_grid = {
    'max_depth': [2, 3, 4],
    'min_samples_leaf': [1, 3, 5, 10, 15],
    'min_samples_split': [2, 5],
    'n_estimators': [100, 250]
}

model = GradientBoostingRegressor()
grid_search = GridSearchCV(model, param_grid, cv=4, verbose=5)
grid_search.fit(X_train, y_train)
print()

print(f'Best params: {grid_search.best_params_}')

print("Best model:")
evaluate_model(grid_search.best_estimator_, X_train, X_test, y_train, y_test)
print()
print("Standart model:")
evaluate_model(GradientBoostingRegressor(), X_train, X_test, y_train, y_test)

Fitting 4 folds for each of 60 candidates, totalling 240 fits
[CV 1/4] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.442 total time=   0.1s
[CV 2/4] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.523 total time=   0.1s
[CV 3/4] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.431 total time=   0.1s
[CV 4/4] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.336 total time=   0.1s
[CV 1/4] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=0.394 total time=   0.2s
[CV 2/4] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=0.500 total time=   0.2s
[CV 3/4] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=0.402 total time=   0.2s
[CV 4/4] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=0.331 total time=   0.2s
[C